In [1]:
from nltk.tokenize import sent_tokenize
text = 'I love this flavor! It\'s by far the best choice and my go-to whenever I go to the grocery store. I wish they would restock it more often though.'
sentence_tokens = sent_tokenize(text)
print(sentence_tokens)

['I love this flavor!', "It's by far the best choice and my go-to whenever I go to the grocery store.", 'I wish they would restock it more often though.']


In [3]:
import os
import pandas as pd
from typing import Dict
import time
from nebula3.gclient.net import Connection
from nebula3.gclient.net.SessionPool import SessionPool
from nebula3.Config import SessionPoolConfig
from nebula3.common.ttypes import ErrorCode
from nebula3.data.ResultSet import ResultSet

In [4]:
os.environ['NEBULA_USER'] = "root"
os.environ['NEBULA_PASSWORD'] = "nebula"
os.environ["GRAPHD_HOST"] = "127.0.0.1"
os.environ["GRAPHD_PORT"] = "9669"
os.environ['NEBULA_ADDRESS'] = "127.0.0.1:9669"
# space_name = "Digital_Safety"
space_name = "test"

In [5]:
config = SessionPoolConfig()

# prepare space
conn = Connection()
conn.open(os.environ["GRAPHD_HOST"], os.environ["GRAPHD_PORT"], 1000)
auth_result = conn.authenticate(os.environ["NEBULA_USER"], os.environ["NEBULA_PASSWORD"])
assert auth_result.get_session_id() != 0
session_pool = SessionPool(os.environ["NEBULA_USER"], os.environ["NEBULA_PASSWORD"], space_name, [(os.environ["GRAPHD_HOST"], os.environ["GRAPHD_PORT"])])
assert session_pool.init(config)

In [6]:
resp = session_pool.execute(
    'USE test;'
    'MATCH ()-[e]->() RETURN e'
)

In [18]:
for row in resp:
    relation_value = row.values()[0].get_value()
    print(relation_value.get_eVal().src)

Value(
    sVal=b'ing\xc3\xa9nieur')
Value(
    sVal=b'I1 \xc2\xab Digital Safety \xc2\xbb')
Value(
    sVal=b'RCO-5.pdf')
Value(
    sVal=b'RCO-5.pdf')
Value(
    sVal=b'RCO-5.pdf')
Value(
    sVal=b'Intelligence Artificielle')
Value(
    sVal=(b'Etat de l\xe2\x80\x99art sur la cartographie automatique et dynamique des'
      b' relations intras et inters documentaires des documents composant un rapport'
      b' de s\xc3\xbbret\xc3\xa9 pour le Projet I1 \xc2\xab Digital Safety \xc2\xbb '
      b'5'))
Value(
    sVal=b'Etat de l_art_RCO-5.pdf')
Value(
    sVal=b'Autorit\xc3\xa9 de S\xc3\xbbret\xc3\xa9 Nucl\xc3\xa9aire ASN')
Value(
    sVal=b'rapport de s\xc3\xbbret\xc3\xa9 nucl\xc3\xa9aire')
Value(
    sVal=b'rapport de s\xc3\xbbret\xc3\xa9 nucl\xc3\xa9aire')
Value(
    sVal=b'logiciel')
Value(
    sVal=(b'3mydnnfyn8s8zjjjbt6rdlpm41920w312xwgvsc1pk3vs30n784Aqd13AqyA1zw9htnf7vvdf289'
      b'tvwcm0v2wj0vwmm8t453kAp8y2tA3xm'))
Value(
    sVal=b'1dh6y1wt95tknsf18z1')


In [11]:
def result_to_dict(result: ResultSet) -> pd.DataFrame:
    """
    build list for each column, and transform to dataframe
    """
    assert result.is_succeeded()
    result_dict = {}
    source = []
    target = []
    relation = []
    for row in result:
        relation_value = row.values()[0].get_value()
        source.append(relation_value.get_eVal().src.get_sVal().decode())
        target.append(relation_value.get_eVal().dst.get_sVal().decode())
        relation.append(relation_value.get_eVal().props[b'relationship'].get_sVal().decode())

    result_dict["source"] = source
    result_dict["target"] = target
    result_dict["relation"] = relation
    return result_dict

In [12]:
result_dict = result_to_dict(resp)

In [20]:
import networkx as nx
from pyvis.network import Network

# Create a directed graph
graph = nx.DiGraph()

# Add nodes and edges to the graph
for src, tgt, rel in zip(result_dict['source'], result_dict['target'], result_dict['relation']):
    graph.add_edge(src, tgt, relation=rel)

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(graph)
# net.show('./example.html')
net.save_graph('./example.html')
